In [1]:
import torch 
import torch.nn as nn
import numpy as np
import struct
from torchvision import models

In [3]:
model2 = models.mobilenet_v2(pretrained=True)
# Aircraft 데이터를 사용하기 위해서 마지막 layer 교체
model2.classifier[1] = torch.nn.Linear(1280, 100, bias = True)

In [10]:
model2.features

Sequential(
  (0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      

In [19]:
for name, param in model2.features.named_parameters():
    print(name, param.shape)
    if len(param.shape) == 1:
        if 'bias' in name:
            print("===bias===")
            print(param.shape)
            print(param)
            
        elif 'wegiht' in name:
            print("===weight===")
            print(param.shape)
            print(param)
            
            
        
        
    
    

0.0.weight torch.Size([32, 3, 3, 3])
0.1.weight torch.Size([32])
0.1.bias torch.Size([32])
===bias===
torch.Size([32])
Parameter containing:
tensor([-8.4354e-02,  5.6023e-01,  3.5002e-01,  2.8363e-01,  9.7327e-01,
         6.4774e-01,  4.9481e-01,  5.5817e-01,  6.1756e-01, -4.2980e-04,
        -3.0858e-01,  9.5334e-01,  4.4609e-01, -3.8414e-01, -9.3045e-04,
         5.7470e-03, -4.2064e-02, -1.7965e-02,  3.3821e-01,  1.1017e-01,
        -2.5284e-01,  5.0251e-01,  3.7990e-01, -1.5532e-02, -4.6869e-01,
         5.1056e-01, -2.8880e-01,  6.4006e-01, -1.0935e-01, -5.9483e-02,
         3.7479e-01,  2.6511e-01])
1.conv.0.0.weight torch.Size([32, 1, 3, 3])
1.conv.0.1.weight torch.Size([32])
1.conv.0.1.bias torch.Size([32])
===bias===
torch.Size([32])
Parameter containing:
tensor([-1.1896e-02,  9.3495e-01,  1.4576e+00,  4.8383e-01,  2.3977e-01,
         1.0311e+00,  9.5301e-01,  4.0770e-01,  5.6844e-01, -4.8085e-03,
         3.6246e-01,  2.0216e-01,  1.0718e+00, -3.6395e-01, -5.7587e-03,
     

In [3]:
beta = []
gamma = []
for name, param in model2.features.named_parameters(): 
    if len(param.shape) == 1 :
        param.requires_grad = False
        if isinstance(param, nn.Parameter):
            if "bias" in name:
                print("=====Beta=====")
                print(param.shape)
                print(param)
                bias = param.tolist()
                beta.append(bias)
            elif "weight" in name:
                print("=====Gamma=====")
                print(param.shape)
                print(param)
                gam = param.tolist()
                gamma.append(gam)

=====Gamma=====
torch.Size([32])
Parameter containing:
tensor([0.0381, 0.1872, 0.1975, 0.2451, 0.1313, 0.1590, 0.0881, 0.2552, 0.0870,
        0.0119, 0.4129, 0.1137, 0.2245, 0.3014, 0.0114, 0.0104, 0.0128, 0.0043,
        0.0668, 0.4108, 0.3578, 0.1278, 0.5250, 0.0039, 0.4444, 0.1298, 0.3284,
        0.2453, 0.3565, 0.3066, 0.4146, 0.1419])
=====Beta=====
torch.Size([32])
Parameter containing:
tensor([-8.4354e-02,  5.6023e-01,  3.5002e-01,  2.8363e-01,  9.7327e-01,
         6.4774e-01,  4.9481e-01,  5.5817e-01,  6.1756e-01, -4.2980e-04,
        -3.0858e-01,  9.5334e-01,  4.4609e-01, -3.8414e-01, -9.3045e-04,
         5.7470e-03, -4.2064e-02, -1.7965e-02,  3.3821e-01,  1.1017e-01,
        -2.5284e-01,  5.0251e-01,  3.7990e-01, -1.5532e-02, -4.6869e-01,
         5.1056e-01, -2.8880e-01,  6.4006e-01, -1.0935e-01, -5.9483e-02,
         3.7479e-01,  2.6511e-01])
=====Gamma=====
torch.Size([32])
Parameter containing:
tensor([0.0258, 0.5501, 0.7154, 0.4909, 1.1534, 0.4905, 0.5998, 0.8480, 0.

Parameter containing:
tensor([ 1.6012e-01, -3.3752e-02,  7.3927e-02,  5.0969e-02,  2.9477e-02,
        -7.5692e-02, -7.6626e-02, -4.0753e-02,  1.4009e-01, -1.5122e-02,
        -1.5760e-02,  9.7933e-02,  1.0181e-01,  3.7009e-02,  3.5030e-02,
        -4.0955e-02,  9.2522e-02, -3.5068e-02,  7.7897e-03, -1.4169e-01,
         1.2789e-01, -3.6193e-03,  6.4323e-02, -2.8132e-02, -6.1017e-02,
        -1.6588e-01,  3.6505e-03, -1.1619e-01, -2.0204e-02, -3.7609e-02,
         6.3429e-02, -2.5834e-02, -5.8903e-02,  1.9019e-01,  1.4742e-01,
        -6.0513e-03,  1.8464e-01, -8.4715e-02,  4.7457e-02,  8.2278e-03,
         1.3186e-02, -9.6777e-02, -3.0021e-02, -1.4381e-02,  3.8873e-03,
        -5.2956e-02, -2.2193e-01, -1.2799e-02, -1.9613e-01,  7.7283e-02,
        -9.9685e-03,  4.2693e-02, -1.4581e-01,  6.3051e-02,  4.3762e-02,
        -3.5985e-02,  4.1143e-02,  8.3583e-02,  1.3670e-01, -4.4257e-02,
         1.1606e-01, -1.1002e-01, -4.2201e-02,  3.3003e-02, -8.0611e-02,
        -1.0153e-02,  1.3474e

In [4]:
import torch
from torch import nn

def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # Use `is_grad_enabled` to determine whether the current mode is training
    # mode or prediction mode
    if not torch.is_grad_enabled():
        # If it is prediction mode, directly use the mean and variance
        # obtained by moving average
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # When using a fully-connected layer, calculate the mean and
            # variance on the feature dimension
            mean = X.mean(dim=0)
            var = ((X - mean)**2).mean(dim=0)
        else:
            # When using a two-dimensional convolutional layer, calculate the
            # mean and variance on the channel dimension (axis=1). Here we
            # need to maintain the shape of `X`, so that the broadcasting
            # operation can be carried out later
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean)**2).mean(dim=(0, 2, 3), keepdim=True)
        # In training mode, the current mean and variance are used for the
        # standardization
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # Update the mean and variance using moving average
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # Scale and shift
    return Y, moving_mean.data, moving_var.data

In [5]:
class BatchNorm(nn.Module):
    # `num_features`: the number of outputs for a fully-connected layer
    # or the number of output channels for a convolutional layer. `num_dims`:
    # 2 for a fully-connected layer and 4 for a convolutional layer
    def __init__(self, num_features, num_dims, gamma, beta):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = nn.Parameter(torch.tensor(gamma))
        self.beta = nn.Parameter(torch.tensor(beta))
        # The variables that are not model parameters are initialized to 0 and 1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # If `X` is not on the main memory, copy `moving_mean` and
        # `moving_var` to the device where `X` is located
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # Save the updated `moving_mean` and `moving_var`
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean, self.moving_var,
            eps=1e-5, momentum=0.9)
        return Y